In [1]:
import os
import numpy as np
import pandas as pd
import time
import shutil


In [2]:
# construct params
tmd = 4090
oh = 0.1778
od = 0.1143

oh_volume = np.pi * ((0.5 * oh) ** 2) * tmd

# geometry -> read base geom and update the entries with new values
geometry = np.loadtxt("base/geometry.txt", delimiter=",", dtype=float)
geometry[:, 1] = od
geometry[:, 2] = oh

# fluids -> read base fluids
fluids = np.loadtxt("base/fluids.txt", delimiter=",", dtype=float)

# numparams -> read base
numparams = np.loadtxt("base/numparams.txt", delimiter=",", dtype=float)

# options -> read base
options = np.loadtxt("base/options.txt", delimiter=",", dtype=float)

# pumping -> read base
pumping = np.loadtxt("base/pumping.txt", delimiter=",", dtype=float)

In [3]:
# create fluids
nn_heavy = np.array([1880, 10.48, 0.91, 0.73, 0.0])
n_light = np.array([1000, 0.0, 0.02, 1.0, 0.0])
nn_light = np.array([1500, 6.40, 0.62, 0.70, 0.0])
n_heavy = np.array([1880, 0.0, 0.02, 1.0, 0.0])

In [ ]:
# define mesh size
# pair => (axial, azimuthal)
az = 20
mesh_grid = {
    "grid_0": [  200, az],
    "grid_1": [ 1022, az],
    "grid_2": [ 2045, az],
    "grid_3": [ 4090, az],
    "grid_4": [ 8180, az],
}

In [5]:
# create pump cases
nn_flow_rate = 1.2 / 60
n_flow_rate = 1.0 / 60

fluid_order = {
    "case_1": np.vstack((n_light, n_heavy)),
    "case_2": np.vstack((n_light, nn_heavy)),
    "case_3": np.vstack((nn_light, n_heavy)),
    "case_4": np.vstack((nn_light, nn_heavy)),
}

# pump schedules
pumping_entries = {
    "case_1": [oh_volume/n_flow_rate, n_flow_rate],
    "case_2": [oh_volume/nn_flow_rate, nn_flow_rate],
    "case_3": [oh_volume/n_flow_rate, n_flow_rate],
    "case_4": [oh_volume/nn_flow_rate, nn_flow_rate],
}


In [7]:
def write_section_parameters(params_directory: str, grid_id: str, flow_case_id: str ):
    
    # get the fluids 
    fluids = fluid_order[flow_case_id]
    
    # update the pump schedule
    pumping = np.array([pumping_entries[flow_case_id] + [0.0, 1.0]])
    
    # update the numparams
    numparams[0] = mesh_grid[grid_id][0]
    numparams[1] = mesh_grid[grid_id][1]

    # do not change the options
    # do not change the geometry

    # save the files to their various locations
    np.savetxt(os.path.join(params_directory, "fluids.txt"), fluids, delimiter=",")
    np.savetxt(os.path.join(params_directory, "geometry.txt"), geometry, delimiter=",")
    np.savetxt(os.path.join(params_directory, "numparams.txt"), numparams, delimiter=",")
    np.savetxt(os.path.join(params_directory, "pumping.txt"), pumping, delimiter=",")
    np.savetxt(os.path.join(params_directory, "options.txt"), options, delimiter=",")

# create the directories
def make_case_directories():
    current_dir = os.getcwd()
    np.savetxt(os.path.join(current_dir, "durations.csv"), np.array([]))

    for grid_key, grid in mesh_grid.items():

        for flow_case in fluid_order.keys():
            case_dir = os.path.join(current_dir, f"{grid_key}-{flow_case}")
            os.makedirs(case_dir, exist_ok=True)

            # create the params/results directories
            params_dir = os.path.join(case_dir, "params")
            results_dir = os.path.join(case_dir, "results")
            os.makedirs(params_dir, exist_ok=True)
            os.makedirs(results_dir, exist_ok=True)

            # write the parameters to the section params dir.
            write_section_parameters(params_dir, grid_key, flow_case)


In [8]:
# invoke.
# always run all cells if anything is changed anywhere

make_case_directories()